In [1]:
import numpy as np
import h5py
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

#IMPORT ALL PRE BUILD FUNCTION 
from func import *
# from sklearn.metrics import accuracy_score
import time
from momentum import *

In [2]:
#PATH SHOULD BE   YOUR_FOLDER_PATH/popular_optimizer/datasets/train_catvnoncat.h5
hf1 = h5py.File('/pythonfile/popular_optimizer/datasets/train_catvnoncat.h5','r')  #READ THE DATASETS
X_train = hf1.get('train_set_x')     #GET THE X TRAINING DATASETS
Y_train = hf1.get('train_set_y')     #GET Y LABEL 

print("Shape of X_train {}  and  Y_train {}".format(X_train.shape,Y_train.shape))

Shape of X_train (209, 64, 64, 3)  and  Y_train (209,)


In [3]:
#flatten the array and normalize 
X_train = np.reshape(X_train,(X_train.shape[0],-1))  #FLATTEN   (209, 64 *64 * 3)
X_train = preprocessing.normalize(X_train)           #NORMALIZING THE DATASETS

Y_train = np.reshape(Y_train,(Y_train.shape[0],1))                # MAKING SURE IT IS IN CORRECT SHAPE

print("Shape of X_train {}  and  Y_train {}".format(X_train.shape,Y_train.shape))

Shape of X_train (209, 12288)  and  Y_train (209, 1)


In [17]:
def model_simple_gradient_descent(X,Y,learning_rate,num_iter,hidden_size,keep_prob):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)

    for i in range(1,num_iter):

        cache, A = model_forward(X, params, L,keep_prob)
        cost = cost_f(A, Y)
        grad = backward(X, Y, params, cache, L,keep_prob)
        params = update_params(params, grad, learning_rate, L)
        if i%20 == 0:
            print('cost of iteration________________', i)
            print(cost)
    return params

In [18]:
params = model_simple_gradient_descent(X_train, Y_train, learning_rate = 0.09,
               num_iter=200, hidden_size=[100, 1],keep_prob=0.5)
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration________________ 20
0.6196480314939471
cost of iteration________________ 40
0.609287022014639
cost of iteration________________ 60
0.585299990570172
cost of iteration________________ 80
0.5867410369882348
cost of iteration________________ 100
0.5573366384458787
cost of iteration________________ 120
0.5481453600680758
cost of iteration________________ 140
0.540081161976517
cost of iteration________________ 160
0.5256660204617498
cost of iteration________________ 180
0.5357365574912608
train_accuracy------------ 0.8086124401913876


In [6]:
def model_momentum_optimizer(X,Y,learning_rate,num_iter,hidden_size,keep_prob):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)
    v = initilization_moment(params)
    for i in range(1,num_iter):
        cache, A = model_forward(X, params, L,keep_prob)
        cost = cost_f(A, Y)
        grad = backward(X, Y, params, cache, L,keep_prob)
        params = update_params_with_momentum(params, grad, v, beta=0.9,learning_rate=learning_rate)
        if i%20 == 0:
            print('cost of iteration________________', i)
            print(cost)
    return params

In [7]:
params = model_momentum_optimizer(X_train, Y_train, learning_rate = 0.09,
               num_iter=200, hidden_size=[100, 1],keep_prob=0.5)
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration________________ 20
0.6144856943253116
cost of iteration________________ 40
0.5715780411001979
cost of iteration________________ 60
0.5438768046959095
cost of iteration________________ 80
0.511577767817021
cost of iteration________________ 100
0.49070902690418217
cost of iteration________________ 120
0.47877285620702753
cost of iteration________________ 140
0.44634502661480824
cost of iteration________________ 160
0.42525388795193764
cost of iteration________________ 180
0.4168274516521236
train_accuracy------------ 0.8516746411483254


In [ ]:
def model_RMSprop_optimizer(X,Y,learning_rate,num_iter,hidden_size,keep_prob):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)
    v = initilization_moment(params)
    for i in range(1,num_iter):
        cache, A = model_forward(X, params, L,keep_prob)
        cost = cost_f(A, Y)
        grad = backward(X, Y, params, cache, L,keep_prob)
        params = update_params_with_RMS(params, grad, v, beta=0.9,learning_rate=learning_rate)
        if i%20 == 0:
            print('cost of iteration________________', i)
            print(cost)
    return params

In [ ]:
params = model_RMSprop_optimizer(X_train, Y_train, learning_rate = 0.09,
               num_iter=200, hidden_size=[100, 1],keep_prob=0.5)
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))